In [1]:
import os
# move 2 directories up
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
# min max scaling
from sklearn.preprocessing import MinMaxScaler
# standard scaler
from sklearn.preprocessing import StandardScaler
from utils.util_functions import MyLinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [6]:
model_dir="models/point/"

In [10]:
model_pt = joblib.load(model_dir+"lgb_next_step_nodiff_12march.pkl")
model_pt_next = joblib.load(model_dir+'lgb_next_step_diff_12march.pkl')
model_lr = joblib.load(model_dir+"lr.pkl")
model_lr_next = joblib.load(model_dir+"lr_next_step.pkl")

In [12]:
df_train = pd.read_csv("data/extra_train.csv", index_col=0)
df_train

,Month,Hour,cons_target,Solar Generation [W/kW],Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],day_year,number,...,net_target,net+1,net-23,net_target+1,net_target-23,net_target_diff,diffuse_solar_radiation+1,direct_solar_radiation+1,relative_humidity+1,drybulb_temp+1
0,8,0,0.441346,0.000000,19.4,81.0,0.000000,0.000000,23,1,...,0.617429,1.411333,2.275800,0.431845,0.506612,-0.029331,0.000000,0.000000,81.0,20.0
1,8,1,0.170776,0.000000,20.0,81.0,0.000000,0.000000,24,1,...,0.431845,0.979417,0.851167,0.394489,0.383397,-0.185584,0.000000,0.000000,87.0,19.4
2,8,2,0.116313,0.000000,19.4,87.0,0.000000,0.000000,25,1,...,0.394489,0.900975,0.834600,0.387705,0.381964,-0.037356,0.000000,0.000000,87.0,19.4
3,8,3,0.106422,0.000000,19.4,87.0,0.000000,0.000000,26,1,...,0.387705,0.922433,0.838167,0.389561,0.382273,-0.006784,0.000000,0.000000,90.0,19.4
4,8,4,0.109127,0.000000,19.4,90.0,0.000000,0.000000,27,1,...,0.389561,0.910483,1.478433,0.388527,0.437649,0.001856,0.000000,0.000000,90.0,18.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43675,7,19,0.802751,31.179167,21.1,81.0,5.389072,5.669881,8754,5,...,0.862123,2.713900,3.463783,0.720845,0.814933,0.090374,3.295837,4.875197,84.0,20.6
43676,7,20,0.552807,4.070833,20.6,84.0,3.295837,4.875197,8755,5,...,0.720845,0.935833,0.457167,0.497751,0.437693,-0.141277,0.000000,0.000000,81.0,21.1
43677,7,21,0.189487,0.000000,21.1,81.0,0.000000,0.000000,8756,5,...,0.497751,2.155017,0.714717,0.650722,0.470007,-0.223094,0.000000,0.000000,79.0,21.7
43678,7,22,0.436347,0.000000,21.7,79.0,0.000000,0.000000,8757,5,...,0.650722,2.316400,1.659000,0.670971,0.588487,0.152971,0.000000,0.000000,76.0,21.3


In [13]:
# get feature names of model_pt_next lgb model
feature_names = model_pt_next.feature_name()
feature_names

['Month',
 'Hour',
 'hour_x',
 'hour_y',
 'month_x',
 'month_y',
 'net_target',
 'net_target-23',
 'net_target_diff',
 'diffuse_solar_radiation+1',
 'direct_solar_radiation+1',
 'relative_humidity+1',
 'drybulb_temp+1']

In [15]:
# get features names of model_lr_next lr model
feature_names_lr = model_lr_next.feature_name()
feature_names_lr

['hour_x',
 'hour_y',
 'month_x',
 'month_y',
 'net_target',
 'net_target-23',
 'diffuse_solar_radiation+1',
 'direct_solar_radiation+1',
 'relative_humidity+1',
 'drybulb_temp+1']

In [17]:
# predict net_target+1 using model_pt_next
preds_lgb = model_pt_next.predict(df_train[feature_names])
preds_lgb

array([0.56556285, 0.39367511, 0.38815161, ..., 0.49855052, 0.58184056,
       0.56444159])

In [18]:
# predict net_target+1 using model_lr_next
preds_lr = model_lr_next.predict(df_train[feature_names_lr])
preds_lr

/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MyLinearRegression was fitted without feature names
  warnings.warn(


array([0.56912697, 0.42178791, 0.39375415, ..., 0.49501992, 0.61609822,
       0.60496253])

In [19]:
# get net_target+1
y = df_train["net_target+1"].values
y

array([0.43184524, 0.39448921, 0.38770487, ..., 0.65072219, 0.67097099,
       0.53245184])

In [21]:
# evaluate predictions using mae and mse metrics
print("MAE lgb:", mean_absolute_error(y, preds_lgb))
print("MSE lgb:", mean_squared_error(y, preds_lgb))
print("MAE lr:", mean_absolute_error(y, preds_lr))
print("MSE lr:", mean_squared_error(y, preds_lr))

MAE lgb: 0.038211742171523656
MSE lgb: 0.004192858234252061
MAE lr: 0.051010902083942794
MSE lr: 0.00543120374987156


In [23]:
# evaluate ensemble predictions using mae and mse metrics
print("MAE ensemble:", mean_absolute_error(y, (preds_lgb+preds_lr)/2))
print("MSE ensemble:", mean_squared_error(y, (preds_lgb+preds_lr)/2))

MAE ensemble: 0.043422427055889994
MSE ensemble: 0.00450799192448922


In [24]:
df_train_2 = pd.read_csv("data/extra_train_phase2.csv", index_col=0)

In [25]:
# make predictions for phase 2
preds_lgb_2 = model_pt_next.predict(df_train_2[feature_names])
preds_lr_2 = model_lr_next.predict(df_train_2[feature_names_lr])
y_2 = df_train_2["net_target+1"].values
# evaluate predictions using mae and mse metrics
print("MAE lgb:", mean_absolute_error(y_2, preds_lgb_2))
print("MSE lgb:", mean_squared_error(y_2, preds_lgb_2))
print("MAE lr:", mean_absolute_error(y_2, preds_lr_2))
print("MSE lr:", mean_squared_error(y_2, preds_lr_2))
# evaluate ensemble predictions using mae and mse metrics
print("MAE ensemble:", mean_absolute_error(y_2, (preds_lgb_2+preds_lr_2)/2))
print("MSE ensemble:", mean_squared_error(y_2, (preds_lgb_2+preds_lr_2)/2))

MAE lgb: 0.04954821767531365
MSE lgb: 0.006145891183799982
MAE lr: 0.05455116342699128
MSE lr: 0.0063285892896925156
MAE ensemble: 0.05056241949932789
MSE ensemble: 0.0059501723140722405


/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MyLinearRegression was fitted without feature names
  warnings.warn(
